In [1]:
%matplotlib inline
import seaborn as sns; 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import gridspec
import scipy
style.use('ggplot')
import datetime
from io import StringIO
import re
import os
import numpy as np
from datetime import datetime
import apt_pkg
apt_pkg.init_system()

# Load installed packages

In [3]:
packages = pd.read_csv('../../../docker_emse/csv/for_analysis/installed_packages.csv',
                      usecols=['package','version','archive','last_updated','date'])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
packages.drop_duplicates(inplace=True)

In [5]:
packages.shape

(2303862, 5)

In [6]:
packages[['package','version','last_updated']].drop_duplicates().shape

(2257988, 3)

In [6]:
for col in packages.columns:
    packages[col] = packages[col].apply(str)

In [7]:
packages.head(2)

,package,version,last_updated,archive,date
0,0ad,0.0.21-2,2018-12-10,stretch,2016-11-19
1,0ad-data,0.0.21-1,2018-12-10,stretch,2016-11-19


# Load debian packages

In [6]:
debian_packages = pd.read_csv('../../../docker_emse/csv/prepared_data/debian_packages_20190905.csv',
                             usecols=['package','version','archive','date'])
debian_packages.drop_duplicates(inplace=True)
debian_packages.shape

(954495, 4)

In [7]:
for col in debian_packages.columns:
    debian_packages[col] = debian_packages[col].apply(str)

In [10]:
debian_packages.rename(columns={'version':'version_c','date':'date_c'}, inplace=True)

In [12]:
debian_packages.tail(2)

,package,version_c,archive,date_c
954493,zzuf,0.15-1+b1,buster,2017-06-19
954494,zzuf,0.15-1+b1,stretch,2017-03-06


In [11]:
packages = (packages
            .merge(debian_packages,
                   on = ['package','archive'],
                   how = 'left'
                  )
           )
packages.shape

(26614399, 7)

In [12]:
# Remove package version that were created after the containers inspection
packages = packages.query('last_updated>=date_c').copy()
packages.shape

(24367357, 7)

In [16]:
packages.head(2)

,package,version,last_updated,archive,date,version_c,date_c
0,0ad,0.0.21-2,2018-12-10,stretch,2016-11-19,0.0.17-1,2015-04-26
1,0ad,0.0.21-2,2018-12-10,stretch,2016-11-19,0.0.18-1,2015-04-27


# Time lag ~ latest available release

In [14]:
time_lag = packages.query('date_c >= date')
time_lag.shape

(3038690, 7)

In [15]:
time_lag = (time_lag
            .sort_values('date_c', ascending=False)
            .groupby(['package','version','archive','last_updated'])
            .first()
            .reset_index()
           )
time_lag.shape

(2262749, 7)

In [16]:
time_lag['date'] = pd.to_datetime(time_lag['date'])
time_lag['date_c'] = pd.to_datetime(time_lag['date_c'])

In [17]:
time_lag['tlag']=time_lag.apply(lambda d: (d['date_c'] - d['date']).days, axis=1)

In [19]:
time_lag.rename(columns={'version_c':'ideal'}, inplace=True)

In [20]:
time_lag = time_lag[['package','version','archive','last_updated','ideal','tlag']]

In [21]:
time_lag.to_csv('../../../docker_emse/csv/for_analysis/tlag_ideal.csv', index=False)

In [22]:
time_lag['compare'] = time_lag.apply(lambda d: compare(d['version'], d['ideal']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Version lag ~ highest available version

In [ ]:
def compare(version,compare_to):
    return apt_pkg.version_compare(str(version), str(compare_to))
    # 1 if version > compare_to
        
packages['compare'] = packages.apply(lambda d: compare(d['version'], d['version_c']), axis=1)
packages.shape

In [26]:
packages.compare.unique()

array([   1,    0,    2,    3,    8,    4,    5,   -1,   -2,   -3,   -4,
          6,   -5,    7,  303,  -16,   16, -303,  -17,   17,   -6,   -7,
         -8,  300,    9,   82,  201,   98,  -98,  184,   11,   -9,  188,
        199,  103,  104,  220, -114, -300,  114,  -14,   15,   14,   13,
         12,   10,   21,   97,  200])

In [27]:
version_lag = packages.query('compare < 0')
version_lag.shape

(783735, 8)

In [28]:
version_lag = (version_lag[['package','version','archive','last_updated','compare']]
               .groupby(['package','version','archive','last_updated'])
               .count()
               .reset_index()
               .rename(columns = {'compare':'vlag'})
              )
version_lag.shape

(374725, 5)

In [29]:
version_lag.to_csv('../../../docker_emse/csv/for_analysis/vlag.csv', index=False)

# Merge between vlag and tlag

In [27]:
version_lag = pd.read_csv('../../../docker_emse/csv/for_analysis/vlag.csv')
version_lag.shape

(374725, 5)

In [28]:
version_lag.head(2)

,package,version,archive,last_updated,vlag
0,adduser,3.115,buster,2017-08-29,1
1,adduser,3.115,buster,2017-09-11,1


In [32]:
time_lag.shape

(2262749, 6)

In [33]:
time_lag = (time_lag
            .merge(version_lag,
                  on =['package','version','archive','last_updated'],
                   how = 'left'
                  )
           )
time_lag.fillna(0, inplace=True)
time_lag.shape

(2262749, 7)

In [39]:
time_lag = time_lag.query('vlag==tlag or vlag != 0')

In [40]:
time_lag.to_csv('../../../docker_emse/csv/for_analysis/tv_lag_ideal.csv', index=False)

In [2]:
lag = pd.read_csv('../../../docker_emse/csv/for_analysis/tv_lag_ideal.csv')

In [24]:
lag.query('last_updated>"2018-05-01" and last_updated < "2018-07-01"').sort_values('tlag',ascending=False).head(40)

,package,version,archive,last_updated,ideal,tlag,vlag
914630,libipc-run-perl,0.92-1,jessie,2018-06-29,0.92-1+deb8u1,1875,1.0
914629,libipc-run-perl,0.92-1,jessie,2018-06-24,0.92-1+deb8u1,1875,1.0
1039787,libmad0,0.15.1b-8,jessie,2018-06-12,0.15.1b-8+deb8u1,1800,1.0
1039786,libmad0,0.15.1b-8,jessie,2018-05-31,0.15.1b-8+deb8u1,1800,1.0
1039783,libmad0,0.15.1b-8,jessie,2018-05-17,0.15.1b-8+deb8u1,1800,1.0
1039784,libmad0,0.15.1b-8,jessie,2018-05-29,0.15.1b-8+deb8u1,1800,1.0
1039785,libmad0,0.15.1b-8,jessie,2018-05-30,0.15.1b-8+deb8u1,1800,1.0
1872038,ncurses-term,5.9-10,jessie,2018-05-16,5.9+20140913-1+deb8u2,1679,6.0
2116595,sensible-utils,0.0.9,jessie,2018-06-07,0.0.9+deb8u1,1649,1.0
2116594,sensible-utils,0.0.9,jessie,2018-06-06,0.0.9+deb8u1,1649,1.0


In [12]:
debian_packages.query('package == "libdjvulibre-text" and version =="3.5.25.4-4"')

,package,version,archive,date
324330,libdjvulibre-text,3.5.25.4-4,jessie,2014-03-26
324331,libdjvulibre-text,3.5.25.4-4,stretch,2015-04-27


In [13]:
debian_packages.query('package == "libdjvulibre-text" and version =="3.5.25.4-4+deb8u1"')

,package,version,archive,date
324332,libdjvulibre-text,3.5.25.4-4+deb8u1,jessie,2019-08-30


In [25]:
debian_packages = pd.read_csv('../../../docker_emse/csv/prepared_data/debian_packages_20190905.csv')
debian_packages.drop_duplicates(inplace=True)
debian_packages.shape

(954495, 6)